In [1]:
cd ../..

C:\Users\thinh\Documents\TN-Vathes\DataJoint\datajoint-python


In [2]:
import os
import datajoint as dj
from pathlib import Path

In [3]:
# set experimental feature to true: DJ_SUPPORT_FILESET_MANAGEMENT
os.environ['DJ_SUPPORT_FILEPATH_MANAGEMENT'] = "TRUE"
os.environ['DJ_SUPPORT_FILESET_MANAGEMENT'] = "TRUE"

In [4]:
# configure an external store
STORE_NAME = 'file-store'
dj.config['stores'] = {STORE_NAME: {
    'protocol': 'file',
    'location': 'D:/dj_store_cache',
    'stage': 'C:/Users/thinh/Documents/Temporary'}
}

In [5]:
schema = dj.Schema('test_fileset')

[2023-02-20 11:04:01,609][INFO]: Connecting root@localhost:3306
[2023-02-20 11:04:03,701][INFO]: Connected root@localhost:3306


In [6]:
@schema
class Session(dj.Manual):
    definition = """
    session_id: int
    ---
    session_fileset: fileset@file-store
    """

In [7]:
data_dir = Path(r"C:\Users\thinh\Documents\Temporary\outbox\report_figures")

In [8]:
# Let check the contents of the `data_dir`
list(data_dir.rglob("*"))

[WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c100r6_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c2000r397_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c2000r408_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c2000r479_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c2000r498_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c2000r503_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c2000r506_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c2000r535_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c2000r538_psychometric_curve.png')]

#### Now, insert as a `fileset` datatype

`fileset` type can be inserted from a directory

In [9]:
Session.insert1((0, data_dir))

In [10]:
Session()

session_id,session_fileset
0,=BLOB=


In [11]:
(Session & {'session_id': 0}).fetch1('session_fileset')

[WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c100r6_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c2000r397_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c2000r408_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c2000r479_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c2000r498_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c2000r503_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c2000r506_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c2000r535_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c2000r538_psychometric_curve.png')]

#### We can also check the contents of the schema-level `fileset` store

In [12]:
schema.fileset['file-store']

fileset_id identifier for a fileset - computed from the aggregated checksum of all files in the fileset,fileset_root the commonpath of all files in the fileset,fileset_size size of the entire fileset in bytes,file_count,fileset_creation_time creation time (UTC) of the fileset
b2af2df8-2481-bde3-4630-585b23767a18,outbox/report_figures,531044,9,2023-02-20 17:04:18


In [13]:
schema.fileset['file-store'].File

fileset_id identifier for a fileset - computed from the aggregated checksum of all files in the fileset,hash hash of filename + contents (attach),"filepath filepath relative to the ""location"" of the store",file
b2af2df8-2481-bde3-4630-585b23767a18,159bdbe3-39a8-bd67-ead7-36176ba7f5bf,outbox/report_figures/m37c100r6_psychometric_curve.png,=BLOB=
b2af2df8-2481-bde3-4630-585b23767a18,64f268ce-e8ba-4e92-bc07-4d32c12d1c24,outbox/report_figures/m37c2000r498_psychometric_curve.png,=BLOB=
b2af2df8-2481-bde3-4630-585b23767a18,97675579-9d34-d714-d212-5aaa05b2c01f,outbox/report_figures/m37c2000r503_psychometric_curve.png,=BLOB=
b2af2df8-2481-bde3-4630-585b23767a18,9b8c4e8c-ed48-04b1-db32-15785cf1b298,outbox/report_figures/m37c2000r408_psychometric_curve.png,=BLOB=
b2af2df8-2481-bde3-4630-585b23767a18,aa8c5918-105d-251a-f492-68c3b3bfa770,outbox/report_figures/m37c2000r397_psychometric_curve.png,=BLOB=
b2af2df8-2481-bde3-4630-585b23767a18,c87263f2-632b-332b-5ca4-40aa7c38161c,outbox/report_figures/m37c2000r535_psychometric_curve.png,=BLOB=
b2af2df8-2481-bde3-4630-585b23767a18,ce5eb1c8-4b8f-a808-106f-7442ba70383f,outbox/report_figures/m37c2000r479_psychometric_curve.png,=BLOB=


In [14]:
schema.external['file-store']

"hash hash of contents (blob), of filename + contents (attach), or relative filepath (filepath)",size size of object in bytes,attachment_name the filename of an attachment,filepath relative filepath or attachment filename,contents_hash used for the filepath datatype,timestamp automatic timestamp
159bdbe3-39a8-bd67-ead7-36176ba7f5bf,74830,m37c100r6_psychometric_curve.png,None,None,2023-02-20 11:04:17
64f268ce-e8ba-4e92-bc07-4d32c12d1c24,69699,m37c2000r498_psychometric_curve.png,None,None,2023-02-20 11:04:18
97675579-9d34-d714-d212-5aaa05b2c01f,63384,m37c2000r503_psychometric_curve.png,None,None,2023-02-20 11:04:18
9b8c4e8c-ed48-04b1-db32-15785cf1b298,48093,m37c2000r408_psychometric_curve.png,None,None,2023-02-20 11:04:17
aa8c5918-105d-251a-f492-68c3b3bfa770,76855,m37c2000r397_psychometric_curve.png,None,None,2023-02-20 11:04:17
c87263f2-632b-332b-5ca4-40aa7c38161c,43068,m37c2000r535_psychometric_curve.png,None,None,2023-02-20 11:04:18
ce5eb1c8-4b8f-a808-106f-7442ba70383f,50216,m37c2000r479_psychometric_curve.png,None,None,2023-02-20 11:04:18


#### Now, let's add another entry into `Session` with the same fileset

In [15]:
Session.insert1((1, data_dir))

In [16]:
Session()

session_id,session_fileset
0,=BLOB=
1,=BLOB=


#### Check the external fileset again, we should not expect file duplications

In [17]:
schema.external['file-store']

"hash hash of contents (blob), of filename + contents (attach), or relative filepath (filepath)",size size of object in bytes,attachment_name the filename of an attachment,filepath relative filepath or attachment filename,contents_hash used for the filepath datatype,timestamp automatic timestamp
159bdbe3-39a8-bd67-ead7-36176ba7f5bf,74830,m37c100r6_psychometric_curve.png,None,None,2023-02-20 11:04:35
64f268ce-e8ba-4e92-bc07-4d32c12d1c24,69699,m37c2000r498_psychometric_curve.png,None,None,2023-02-20 11:04:35
97675579-9d34-d714-d212-5aaa05b2c01f,63384,m37c2000r503_psychometric_curve.png,None,None,2023-02-20 11:04:35
9b8c4e8c-ed48-04b1-db32-15785cf1b298,48093,m37c2000r408_psychometric_curve.png,None,None,2023-02-20 11:04:35
aa8c5918-105d-251a-f492-68c3b3bfa770,76855,m37c2000r397_psychometric_curve.png,None,None,2023-02-20 11:04:35
c87263f2-632b-332b-5ca4-40aa7c38161c,43068,m37c2000r535_psychometric_curve.png,None,None,2023-02-20 11:04:35
ce5eb1c8-4b8f-a808-106f-7442ba70383f,50216,m37c2000r479_psychometric_curve.png,None,None,2023-02-20 11:04:35


In [18]:
schema.fileset['file-store']

fileset_id identifier for a fileset - computed from the aggregated checksum of all files in the fileset,fileset_root the commonpath of all files in the fileset,fileset_size size of the entire fileset in bytes,file_count,fileset_creation_time creation time (UTC) of the fileset
b2af2df8-2481-bde3-4630-585b23767a18,outbox/report_figures,531044,9,2023-02-20 17:04:18


In [19]:
schema.fileset['file-store'].File

fileset_id identifier for a fileset - computed from the aggregated checksum of all files in the fileset,hash hash of filename + contents (attach),"filepath filepath relative to the ""location"" of the store",file
b2af2df8-2481-bde3-4630-585b23767a18,159bdbe3-39a8-bd67-ead7-36176ba7f5bf,outbox/report_figures/m37c100r6_psychometric_curve.png,=BLOB=
b2af2df8-2481-bde3-4630-585b23767a18,64f268ce-e8ba-4e92-bc07-4d32c12d1c24,outbox/report_figures/m37c2000r498_psychometric_curve.png,=BLOB=
b2af2df8-2481-bde3-4630-585b23767a18,97675579-9d34-d714-d212-5aaa05b2c01f,outbox/report_figures/m37c2000r503_psychometric_curve.png,=BLOB=
b2af2df8-2481-bde3-4630-585b23767a18,9b8c4e8c-ed48-04b1-db32-15785cf1b298,outbox/report_figures/m37c2000r408_psychometric_curve.png,=BLOB=
b2af2df8-2481-bde3-4630-585b23767a18,aa8c5918-105d-251a-f492-68c3b3bfa770,outbox/report_figures/m37c2000r397_psychometric_curve.png,=BLOB=
b2af2df8-2481-bde3-4630-585b23767a18,c87263f2-632b-332b-5ca4-40aa7c38161c,outbox/report_figures/m37c2000r535_psychometric_curve.png,=BLOB=
b2af2df8-2481-bde3-4630-585b23767a18,ce5eb1c8-4b8f-a808-106f-7442ba70383f,outbox/report_figures/m37c2000r479_psychometric_curve.png,=BLOB=


#### Next, let's introduce a new fileset - we'll use the same files from "data_dir" plus one more file

In [20]:
data_dir

WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures')

In [21]:
list(Path(data_dir.parent).glob("*"))

[WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures'),
 WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/structure.oebin')]

In [22]:
fileset2 = list(data_dir.rglob("*")) + [Path('C:/Users/thinh/Documents/Temporary/outbox/structure.oebin')]

In [23]:
fileset2

[WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c100r6_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c2000r397_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c2000r408_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c2000r479_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c2000r498_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c2000r503_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c2000r506_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c2000r535_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c2000r538_psychometric_curve.png'),
 WindowsPath(

#### Insert new record into Session - `fileset` type can be inserted as a list of paths

In [29]:
Session.insert1((2, fileset2))

In [30]:
Session()

session_id,session_fileset
0,=BLOB=
1,=BLOB=
2,=BLOB=


#### Let's check the external fileset store again

In [31]:
schema.fileset['file-store']

fileset_id identifier for a fileset - computed from the aggregated checksum of all files in the fileset,fileset_root the commonpath of all files in the fileset,fileset_size size of the entire fileset in bytes,file_count,fileset_creation_time creation time (UTC) of the fileset
b2af2df8-2481-bde3-4630-585b23767a18,outbox/report_figures,531044,9,2023-02-20 17:04:18
c0d1d460-bc3c-c172-e31b-7a0e0bd547f5,outbox,1851506,10,2023-02-20 17:05:09


In [32]:
schema.fileset['file-store'].File

fileset_id identifier for a fileset - computed from the aggregated checksum of all files in the fileset,hash hash of filename + contents (attach),"filepath filepath relative to the ""location"" of the store",file
b2af2df8-2481-bde3-4630-585b23767a18,159bdbe3-39a8-bd67-ead7-36176ba7f5bf,outbox/report_figures/m37c100r6_psychometric_curve.png,=BLOB=
b2af2df8-2481-bde3-4630-585b23767a18,64f268ce-e8ba-4e92-bc07-4d32c12d1c24,outbox/report_figures/m37c2000r498_psychometric_curve.png,=BLOB=
b2af2df8-2481-bde3-4630-585b23767a18,97675579-9d34-d714-d212-5aaa05b2c01f,outbox/report_figures/m37c2000r503_psychometric_curve.png,=BLOB=
b2af2df8-2481-bde3-4630-585b23767a18,9b8c4e8c-ed48-04b1-db32-15785cf1b298,outbox/report_figures/m37c2000r408_psychometric_curve.png,=BLOB=
b2af2df8-2481-bde3-4630-585b23767a18,aa8c5918-105d-251a-f492-68c3b3bfa770,outbox/report_figures/m37c2000r397_psychometric_curve.png,=BLOB=
b2af2df8-2481-bde3-4630-585b23767a18,c87263f2-632b-332b-5ca4-40aa7c38161c,outbox/report_figures/m37c2000r535_psychometric_curve.png,=BLOB=
b2af2df8-2481-bde3-4630-585b23767a18,ce5eb1c8-4b8f-a808-106f-7442ba70383f,outbox/report_figures/m37c2000r479_psychometric_curve.png,=BLOB=


In [33]:
schema.external['file-store']

"hash hash of contents (blob), of filename + contents (attach), or relative filepath (filepath)",size size of object in bytes,attachment_name the filename of an attachment,filepath relative filepath or attachment filename,contents_hash used for the filepath datatype,timestamp automatic timestamp
159bdbe3-39a8-bd67-ead7-36176ba7f5bf,74830,m37c100r6_psychometric_curve.png,None,None,2023-02-20 11:06:23
461628a8-0a29-3e98-a114-45cef0581998,1320462,structure.oebin,None,None,2023-02-20 11:06:24
64f268ce-e8ba-4e92-bc07-4d32c12d1c24,69699,m37c2000r498_psychometric_curve.png,None,None,2023-02-20 11:06:23
97675579-9d34-d714-d212-5aaa05b2c01f,63384,m37c2000r503_psychometric_curve.png,None,None,2023-02-20 11:06:23
9b8c4e8c-ed48-04b1-db32-15785cf1b298,48093,m37c2000r408_psychometric_curve.png,None,None,2023-02-20 11:06:23
aa8c5918-105d-251a-f492-68c3b3bfa770,76855,m37c2000r397_psychometric_curve.png,None,None,2023-02-20 11:06:23
c87263f2-632b-332b-5ca4-40aa7c38161c,43068,m37c2000r535_psychometric_curve.png,None,None,2023-02-20 11:06:24


#### No duplication of files on external store (thanks to the feature of "attach" type)

In [34]:
(Session & "session_id = 2").fetch1('session_fileset')

[WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c100r6_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c2000r397_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c2000r408_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c2000r479_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c2000r498_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c2000r503_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c2000r506_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c2000r535_psychometric_curve.png'),
 WindowsPath('C:/Users/thinh/Documents/Temporary/outbox/report_figures/m37c2000r538_psychometric_curve.png'),
 WindowsPath(